# Summary

Runs [fasttext's](https://fasttext.cc/) word2vec on n-days of reading sessions

### Imports etc.

In [26]:
import copy
import csv
import datetime
import math

import numpy as np
import pandas as pd
import os,sys
import json
import time

import fasttext
print('ready')

ready


## Prepare data

In [27]:
# ## select which files to train on
day_start = datetime.date(2020,4,27)
n_days = 7
dt = 3600
nlen_min = 2
nlen_max = 30
ids = 'qid'
lang = 'wikidata'

# ## select which files to train on
# day_start = datetime.date(2020,4,1)
# n_days = 5
# dt = 3600
# nlen_min = 2
# nlen_max = 30
# ids = 'pageid'
# lang = 'simplewiki'



In [28]:
## get all the files
PATH_FILES = '../output/sessions'
PATH_OUT = '../output/models'
day_start_str = day_start.strftime("%Y-%m-%d")#str(datetime.date(year,month,day))
list_filenames = []
for i in range(n_days):
    date_object = day_start+datetime.timedelta(days=i)
    day_str = date_object.strftime("%Y-%m-%d")#str(datetime.date(year,month,day))
    filename = os.path.join(
        PATH_FILES,
        'sessions-filtered-%s_%s_%s_dt%s_nmin%s_nmax%s'%(ids,lang,day_str,dt,nlen_min,nlen_max)
    )
    list_filenames += [filename]
print(len(list_filenames))

5


In [29]:
## create a single training file for fasttext
## this file will be deledted after training
from multiprocessing import Pool
n_cores_max = 20 ## use at most this many cores to read the individual files when creating a single file

PATH_tmp = '../tmp/'
if not os.path.exists(PATH_tmp):
    os.makedirs(PATH_tmp)
FILE_tmp = os.path.join(PATH_tmp,'tmp_corpus.train')
limit = -1 ## number of lines per file (-1 if all)
def make_tmp_corpus_from_files(filename_in):
    with open(FILE_tmp,'a') as fout:
        i = 0
        for line in open(filename_in):
            i+=1
            fout.write(line)
            if i==limit:
                break
    return None

## append files into single file for fasttext
n_files = len(list_filenames)
with open(FILE_tmp,'w') as fin:
    fin.write('')
try:
    pool = Pool( min([n_files,n_cores_max]))
    pool.map(make_tmp_corpus_from_files,list_filenames)
finally:
    pool.close()
    pool.join()

In [30]:
i=0
for line in open(FILE_tmp):
    i+=1
i

131889

## Train

In [31]:
## Parameters
mode = 'cbow' ## (if 1: skip-gram, else cbow)
size = 50 ## number of dimensions
window = 5 ## context window size
sample = 0.001 ## downsample high-frequency words
negative = 5 ##negative sampling (noise words)
min_count = 1 ## words with less occurrences in total will be ignored
epochs  = 5 ## number of iterations
workers = 20 ## number of cores to use


In [32]:
## train the model
t1=time.time()
model = fasttext.train_unsupervised(
    FILE_tmp, 
    dim = size, epoch=epochs, model=mode,minCount=min_count,ws=window,neg=negative, t=sample,
    thread=workers,
    maxn=0 ## no subword
)
t2=time.time()
print(t2-t1)

## remove tmp corpus file
os.system('rm -f %s'%FILE_tmp)

0.9956035614013672


0

In [34]:
filename_save = os.path.join(
    PATH_OUT,
    'word2vec_sessions-filtered-%s_%s_%s_%s_dt%s_nmin%s_nmax%s_params-%s-%s-%s-%s-%s-%s-%s.bin'%(
        ids,lang,day_start_str,n_days,dt,nlen_min,nlen_max,
        mode,size,window,sample,negative,min_count,epochs
    )
)
model.save_model(filename_save)

In [35]:
len(model.get_words())

55910